In [175]:
import cv2 as cv
print("Current version of openCV=", cv.__version__)
import numpy as np
import os
import glob

Current version of openCV= 4.1.1


In [176]:
left_calibration_path = './calibration/fisheye_calibrated/{}.png'
right_calibration_path ='./calibration/pinhole_calibrated/{}.png'

In [177]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 6
points_per_column = 10
objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in Right image plane.
imgpointsR = [] # 2d points in Left image plane.
imagesL = glob.glob(left_calibration_path.format('*'))
imagesR = glob.glob(right_calibration_path.format('*'))
for img_id in range(len(imagesR)):
    fnameL = left_calibration_path.format(img_id)
    fnameR = right_calibration_path.format(img_id)
    imgL = cv.imread(fnameL)
    imgR = cv.imread(fnameR)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, (points_per_row,points_per_column), None)
    retR, cornersR = cv.findChessboardCorners(grayR, (points_per_row,points_per_column), None)
    # If found on both, add object points, image points (after refining them)
    if retL == True and retR == True:
        objpoints.append(objp)
        corners2L = cv.cornerSubPix(grayL,cornersL, (11,11), (-1,-1), criteria)
        corners2R = cv.cornerSubPix(grayR,cornersR, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)
        imgpointsR.append(cornersR)
        # Draw and display the corners
        cv.drawChessboardCorners(imgL, (points_per_row,points_per_column), corners2L, retL)
        cv.drawChessboardCorners(imgR, (points_per_row,points_per_column), corners2R, retR)
        cv.imshow('img', imgL)
        cv.waitKey(500)
        cv.imshow('img', imgR)
        cv.waitKey(500)
cv.destroyAllWindows()

In [241]:
img = cv.imread(left_calibration_path.format(img_id))
width, height = img.shape[:-1]
R = None
T = None
E = None
F = None
stereocalib_criteria = (cv.TERM_CRITERIA_MAX_ITER | cv.TERM_CRITERIA_EPS, 100, 1e-5)
stereocalib_flags = cv.CALIB_RATIONAL_MODEL | \
cv.CALIB_SAME_FOCAL_LENGTH | \ #This flag decreases the accuracy but now the results match better... it is probably better for the 3d reconstruct to be on
#cv.CALIB_ZERO_TANGENT_DIST 
#cv.CALIB_FIX_ASPECT_RATIO 
#cv.CALIB_FIX_K3 | \
#cv.CALIB_FIX_K4 | \
#cv.CALIB_FIX_K5 | \
print(width, height)
retval, KL, DL, KR, DR, R, T, E, F = \
 cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR,
                                #imageSize = (width, height),
                                imageSize = (height, width),
                                cameraMatrix1 = None,
                                distCoeffs1 = None,
                                cameraMatrix2 = None,
                                distCoeffs2 = None,
                                R=R,
                                T=T,
                                E=E,
                                F=F,
                                #flags=cv.CALIB_FIX_INTRINSIC
                                flags=stereocalib_flags,
                                criteria=stereocalib_criteria
                   )

720 1280


In [242]:
print(retval)
print("cameraM1: \n", KL)
print("distCoeffs1: \n", DL)
print("cameraM2: \n", KR)
print("distCoeffs2: \n", DR)
print("R: ", R)
print("T: ", T)
print("E: ", E)
print("F: ", F)

3.1575666183879867
cameraM1: 
 [[792.26793919   0.         706.58041355]
 [  0.         792.59631878 380.73597543]
 [  0.           0.           1.        ]]
distCoeffs1: 
 [[-6.37939819e+00  2.52521642e+01 -6.39684047e-03 -5.36138676e-03
  -1.85070503e+01 -6.36311560e+00  2.52641441e+01 -1.84641289e+01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
cameraM2: 
 [[792.26793919   0.         643.12858763]
 [  0.         792.59631878 329.23295824]
 [  0.           0.           1.        ]]
distCoeffs2: 
 [[ 1.05283236e+02  4.85168110e+02 -4.71871195e-03  5.59023243e-03
   2.90898499e+02  8.87752341e+01  4.45286377e+02  2.48158199e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
R:  [[ 9.99643316e-01 -6.45007263e-03 -2.59159643e-02]
 [ 6.47184023e-03  9.99978772e-01  7.56141002e-04]
 [ 2.59105370e-02 -9.23595279e-04  9.99663839e-01]]
T:  [[ 3.62839261]
 [-0.09282841]
 [ 0.02719998

In [243]:
img_size = imgL.shape[:2]
alpha = 0 ### =0 crop, =1 keep dimensions
print(width, height)

RL = np.zeros(shape=(3,3))
RR = np.zeros(shape=(3,3))
PL = np.zeros(shape=(3,4))
PR = np.zeros(shape=(3,4))

"""out = cv.stereoRectify(KL, DL, KR, DR, (width, height), R=R, T=T, 
                       R1=RL, R2=RR, P1=PL, P2=PR,
                      Q=None, flags=cv.CALIB_ZERO_DISPARITY, alpha=alpha, newImageSize=(0,0))
"""
RL, RR, PL, PR, Q, roiL, roiR = cv.stereoRectify(KL, 
                                                 DL, 
                                                 KR, 
                                                 DR, 
                                                 (height, width), 
                                                 R=R, 
                                                 T=T, 
                                                 alpha=alpha)

720 1280


In [244]:
print("RL: \n", RL)
print("RR: \n", RR)
print("PL: \n", PL)
print("RR: \n", PR)
print("Q: ", Q)
print("roiL: \n", roiL)
print("roiR: \n", roiR)


RL: 
 [[ 9.99316917e-01 -3.20289673e-02 -1.84348513e-02]
 [ 3.20334307e-02  9.99486797e-01 -5.31985776e-05]
 [ 1.84270944e-02 -5.37369295e-04  9.99830062e-01]]
RR: 
 [[ 9.99644823e-01 -2.55748073e-02  7.49376547e-03]
 [ 2.55766212e-02  9.99672854e-01 -1.46298590e-04]
 [-7.48757236e-03  3.37911829e-04  9.99971911e-01]]
PL: 
 [[1.10473981e+03 0.00000000e+00 6.54905334e+02 0.00000000e+00]
 [0.00000000e+00 1.10473981e+03 3.33511375e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
RR: 
 [[1.10473981e+03 0.00000000e+00 6.54905334e+02 4.00985395e+03]
 [0.00000000e+00 1.10473981e+03 3.33511375e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
Q:  [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -6.54905334e+02]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -3.33511375e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.10473981e+03]
 [ 0.00000000e+00  0.00000000e+00 -2.75506245e-01  0.00000000e+00]]
roiL: 
 (0, 0, 

In [245]:
print("Applying Undistort\n")
#map1x, map1y = cv.initUndistortRectifyMap(KL, DL, RL, PL, (width, height), cv.CV_32FC1)
#map2x, map2y = cv.initUndistortRectifyMap(KR, DR, RR, PR, (width, height), cv.CV_32FC1)

map1x, map1y = cv.initUndistortRectifyMap(KL, DL, RL, PL, (height, width), cv.CV_32FC1)
map2x, map2y = cv.initUndistortRectifyMap(KR, DR, RR, PR, (height, width), cv.CV_32FC1)
print("Undistort complete\n")

"""
for img_id in range(len(imagesR)):
    fnameL = left_calibration_path.format(img_id)
    fnameR = right_calibration_path.format(img_id)
    print("fname: ", fnameL, fnameR)
    imgL = cv.imread(fnameL)
    imgR = cv.imread(fnameR)
    imgU1 = np.zeros((height,width,3), np.uint8)
    imgU1 = cv.remap(imgL, map1x, map1y, cv.INTER_LINEAR, imgU1, cv.BORDER_CONSTANT, 0)
    imgU2 = cv.remap(imgR, map2x, map2y, cv.INTER_LINEAR)
    cv.imshow("imageL", imgL);
    cv.imshow("imageR", imgR);
    cv.imshow("image1L", imgU1);
    cv.imshow("image2R", imgU2);
    k = cv.waitKey(5);
    if(k==27):
        break;
"""
print("Maps1: \n", map1x, map1y)
print("Maps2: \n", map2x, map2y)

#image remap example
img_id = 1
fnameL = left_calibration_path.format(img_id)
fnameR = right_calibration_path.format(img_id)
print("fname: ", fnameL, fnameR)
imgL = cv.imread(fnameL)
imgR = cv.imread(fnameR)
cv.imshow('imgL', imgL)
cv.waitKey(500)

left_img_remap = cv.remap(imgL, map1x, map1y, cv.INTER_LANCZOS4)
right_img_remap = cv.remap(imgR, map2x, map2y, cv.INTER_LANCZOS4)

Applying Undistort

Undistort complete

Maps1: 
 [[ 246.81093  247.52002  248.22914 ... 1157.3815  1158.0944  1158.807  ]
 [ 246.84035  247.54944  248.25856 ... 1157.4027  1158.1155  1158.8282 ]
 [ 246.86978  247.57887  248.288   ... 1157.4238  1158.1366  1158.8494 ]
 ...
 [ 266.6991   267.402    268.10498 ... 1169.6444  1170.3501  1171.0558 ]
 [ 266.7254   267.4283   268.13126 ... 1169.657   1170.3627  1171.0682 ]
 [ 266.75174  267.45462  268.15753 ... 1169.6696  1170.3751  1171.0807 ]] [[156.25671  156.23654  156.21638  ... 124.65127  124.62217  124.59307 ]
 [156.97183  156.95168  156.93153  ... 125.37354  125.34443  125.31533 ]
 [157.68694  157.6668   157.64665  ... 126.095795 126.06668  126.037575]
 ...
 [662.9155   662.9015   662.8875   ... 636.3198   636.2916   636.26337 ]
 [663.61237  663.5984   663.58435  ... 637.0231   636.9948   636.96655 ]
 [664.30914  664.2952   664.28125  ... 637.72626  637.69806  637.6698  ]]
Maps2: 
 [[ 102.09547   102.88953   103.68356  ... 1136.2217   

In [249]:
cv.imshow("imgL remap", left_img_remap)
cv.imshow("imgR remap", right_img_remap)
cv.waitKey(500)
cv.destroyAllWindows()